# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

In [80]:
import requests

url = 'https://en.wikipedia.org/wiki/Data_science'

In [81]:
response = requests.get(url).content

## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [82]:
from bs4 import BeautifulSoup as bs

In [83]:
soup = bs(response)

all_links = soup.find_all('a', href = True)

# Find links    

# Only external links
# ext_links = [link.get("href") for link in soup("a") if "http" in link.get("href")]  


# link1
# links = [link for link in soup.find_all('a')]

# links = soup.findAll('a', attrs={'href': re.compile("^http://")})
# links2 = soup.findAll('a', attrs={'href': re.compile("^//")})
# links.get('href')
# print(links)
# empty1 = []

# for i in links:
#     if '%' in i:
#         empty1.append(i)
        

# empty2 = []
        
# for i in links2:
#     if '%' not in i:
#         empty2.append(i)
        

        
# empty1    

## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relativel Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [84]:
domain = 'http://wikipedia.org'

In [87]:
# 


# absolute_links = [link for link in all_links if link.startswith('http') and '%' not in link]

# absolute_links = [link for link in all_links if link.startswith('http') and '%' not in link]
relative_links = [domain + link for link in all_links if link.startswith('/') and '%' not in link]
# combine_links = absolute_links
# combine_links.extend(x for x in relative_links if x not in combine_links)
# combine_links
# relativel_links = [domain + link for link in all_links if link.startswith('/') and '%' not in link]

# clean = []
# for i in all_links:
#     if '%' in i:
#         clean.append(i)
        
# clean
#         clean.append(i)
# clean
# relativel_links = re.findall('"((domain + (http|ftp)s?://.*?))"', str(soup))

# combined_list = absolute_links + relativel_links

# print(combined_list)

TypeError: 'NoneType' object is not callable

## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [89]:
import os

In [104]:
# path = '/Users/anielkaaslan/Documents/wikipedia'

# os.mkdir(path, 775 ) 

def main():
    
    '''
        Get Current working Directory
    '''
    currentDirectory = os.getcwd()
    
    print(currentDirectory)
    
    '''
        Create WIKIPEDIA working Directory
    '''
    os.mkdir('/wikipedia')
    
    '''
        Change to WIKIPEDIA working Directory
    '''
    
    os.chdir('/wikipedia')
    
    ''' 
        Get current working directory
    
    '''
    
currentDirectory = os.getcwd()
    
print(currentDirectory)
        

# cwd = os.getcwd() 

# cwd
# if not os.path.exists(dir):
#     os.mkdir(dir)

# def createFolder(directory):
#     try:
#         if not os.path.exists(directory):
#             os.makedirs(directory)
#     except OSError:
#         print ('Error: Creating directory. ' +  directory)
        

# # Example
# createFolder('./Desktop/wikipedia/')

/Users/anielkaaslan/Documents/data-labs/module-1/lab-parallelization/your-code


## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [ ]:
from slugify import slugify

In [ ]:
def index_page()

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [ ]:
import multiprocessing